**[SQLite](https://www.sqlite.org/)** is one of the most common database engines, and has many advantages:
* The database is stored in a single file, making it portable.
* You can use a SQLite database directly from Python, and don't need a separate program running.
* It implements most SQL commands, enabling you to use most of the statements you're familiar with.

However, particularly when developing larger applications, **SQLite** has a few downsides that make other database engines more attractive:
* Only one process at a time can write to the database. When you have a complex web application, you may have multiple processes updating information in the database at the same time. For example, on Facebook, one process might handle updating user information, and another might handle generating the news feed.
* You can't take advantage of performance features, such as caching. Because a SQLite database is a single file, and it doesn't require a special program to run, it can't have performance optimizations like caching. When running a site like Facebook that has a ton of traffic, it's important to be able to lookup data quickly.
* SQLite doesn't have any built-in security. With a production website, it's common to want some people to be able to modify tables in a database (write), and others to only be able to make SELECT queries to tables in the database (read). This is because giving someone write access to the database can be a security risk, in that they can update or overwrite data. SQLite doesn't allow for restricting access to a database in this way.

In general, **SQLite** is good in cases where having a small and simple database engine is important. **SQLite** is used extensively in embedded applications, such as Android and iOS applications.

In cases where there will be multiple users or performance is important, **[PostgreSQL](http://www.postgresql.org/)** is the most commonly used database engine.

At a high level, **PostgreSQL** consists of two pieces, a server and clients.

The server is a program that manages databases and handles queries. Clients communicate back and forth to the server. Only the server ever directly accesses the databases -- the clients can only make requests to the server.

One of the advantages of this model is that multiple clients can communicate with the server at the same time. This allows multiple processes to write to a database at the same time.

It's possible to run a **PostgreSQL server** either remotely or locally. If it's remote, you connect to it via the internet. If it's local, you connect to it on your own machine. In both cases, you'll be connecting to **PostgreSQL** via a system port.

By default, **PostgreSQL** uses **port 5432** to communicate with the outside world. If you start a PostgreSQL server, it will listen for incoming connections on **port 5432**. Clients will be able to connect to the server using this port. If you start a client, you'll have to specify which server to connect to, along with the port to connect to.

There are many clients for **[PostgreSQL](http://www.postgresql.org/)**, including [graphical clients](https://wiki.postgresql.org/wiki/Community_Guide_to_PostgreSQL_GUI_Tools).

The most common Python client for **PostgreSQL** is called [psycopg2](http://initd.org/psycopg/).

$ **pip install psycopg2**  
Collecting psycopg2  
  Downloading psycopg2-2.7.3.1-cp36-cp36m-macosx_10_6_intel.macosx_10_9_intel.macosx_10_9_x86_64.macosx_10_10_intel.macosx_10_10_x86_64.whl (1.5MB)  
    100% |████████████████████████████████| 1.5MB 6.6kB/s  
Installing collected packages: psycopg2  
Successfully installed psycopg2-2.7.3.1

In [1]:
# Import the psycopg2 library.
import psycopg2

When PostgreSQL is first installed, the default user account is called **postgres**, with an associated database called **postgres**.

**psycopg2** will default to connecting to **port 5432** on the current computer.

In [2]:
# Connect to the postgres database with the user postgres.
conn = psycopg2.connect("dbname=postgres user=postgres")

In [3]:
# Initialize a Cursor object from the connection.
cur = conn.cursor()

In [4]:
# Use the print function to display the Cursor object.
print(cur)

<cursor object at 0x1076d6428; closed: 0>


When you're done with a Connection object, you should close it to avoid issues where one connection prevents another from executing a query.

In [5]:
# Close the Connection using the close method.
conn.close()

Once we've connected to a database, we can create a table inside that database.

In [6]:
# Connect to the lijia database as the user lijia
conn = psycopg2.connect("dbname=lijia user=lijia")
cur = conn.cursor()

In [7]:
# A SQL query that creates a table called notes in the lijia database.
cur.execute('CREATE TABLE notes(\
id integer PRIMARY KEY,\
body text,\
title text\
);')

In [8]:
# Close the Connection using the close method.
conn.close()

Whenever we open a Connection in **psycopg2**, a new transaction will automatically be created. All queries run up until the **[commit](http://initd.org/psycopg/docs/connection.html#connection.commit)** method is called will be placed into the same transaction block. When **commit** is called, the PostgreSQL engine will run all the queries at once.

If we don't want to apply the changes in the transaction block, we can call the **[rollback](http://initd.org/psycopg/docs/connection.html#connection.rollback)** method to remove the transaction. Not calling either **commit** or **rollback** will cause the transaction to stay in a pending state, and will result in the changes not being applied to the database.

In [9]:
# Connect to the lijia database as the user lijia.
conn = psycopg2.connect("dbname=lijia user=lijia")

cur = conn.cursor()

# A SQL query that creates a table called notes in the lijia database.
cur.execute('CREATE TABLE notes(\
id integer PRIMARY KEY,\
body text,\
title text\
);')

# Use the commit method on the Connection object to apply the changes in the transaction to the database.
conn.commit()

# Close the Connection.
conn.close()

There are cases when you won't want to manage a transaction, and you'll instead want changes right away. This is most common when you're making changes to the database that you want to be guaranteed to happen immediately.

To activate autocommit mode, we'll need to set the **[autocommit](http://initd.org/psycopg/docs/connection.html#connection.autocommit)** property of the Connection object to **True**.

In [10]:
# Connect to the lijia database as the user lijia.
conn = psycopg2.connect("dbname=lijia user=lijia")

# Set the autocommit property of the Connection object to True.
conn.autocommit = True

cur = conn.cursor()

# A SQL query that creates a table called facts in the lijia database.
cur.execute('CREATE TABLE facts(\
id integer PRIMARY KEY,\
country text,\
value integer\
);')

# Close the Connection.
conn.close()

We can issue SELECT queries against a database using the **execute** method, along with the **[fetchall](http://initd.org/psycopg/docs/cursor.html#cursor.fetchall)** and **[fetchone](http://initd.org/psycopg/docs/cursor.html#cursor.fetchone)** methods to retrieve results.

In [11]:
# Connect to the lijia database as the user lijia.
conn = psycopg2.connect("dbname=lijia user=lijia")

cur = conn.cursor()

# Execute a SQL query that inserts a row into the notes table.
cur.execute('''INSERT INTO notes(\
VALUES (1, 'Do more missions on Dataquest.', 'Dataquest reminder')\
);''')

# Execute a SQL query that selects all of the rows from the notes table.
cur.execute('SELECT * FROM notes;')

# Fetch all of the results and print them out.
rows = cur.fetchall()
print(rows)

# Close the Connection.
conn.close()

[(1, 'Do more missions on Dataquest.', 'Dataquest reminder')]


One of the most powerful aspects of PostgreSQL is that it enables you to create multiple databases. Different databases are generally used to hold information about different applications.

We can create a database using the **CREATE DATABASE** SQL statement. We can specify the user who will own the database when we create it as well, using the **OWNER** statement.

In [12]:
# Connect to the lijia database with the user lijia.
conn = psycopg2.connect("dbname=lijia user=lijia")

# Set the connection to autocommit mode.
conn.autocommit = True

cur = conn.cursor()

# Create a database called income where the owner is the user lijia.
cur.execute('CREATE DATABASE income OWNER lijia;')

# Close the Connection.
conn.close()

We can delete a database using the **DROP DATABASE** statement. The **DROP DATABASE** statement will immediately remove a database, provided the user executing the query has the right permissions. It should be used with caution when working with real data.

In [13]:
# Connect to the lijia database with the user lijia.
conn = psycopg2.connect("dbname=lijia user=lijia")

# Set the connection to autocommit mode.
conn.autocommit = True

cur = conn.cursor()

# Drop the income database.
cur.execute('DROP DATABASE income;')

# Close the Connection.
conn.close()